In [1]:
"""
Домашнее задание. Свёрточные сети
Здесь вам предстоит построить и обучить свою первую свёрточную сеть для классификации изображений на данных CIFAR10.
"""

import tensorflow as tf
from tqdm import tqdm_notebook
import numpy as np
# from keras.datasets import cifar10
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

ImportError: cannot import name 'keras' from 'tensorflow' (unknown location)

In [2]:
# Загрузите данные, разделите их на обучающую и тестовую выборки. Размер тестовой выборки должен быть  10**4
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=10**4, random_state=42)

NameError: name 'cifar10' is not defined

In [ ]:
class_names = np.array(['airplane','automobile ','bird ','cat ','deer ','dog ','frog ','horse ','ship ','truck']) # deer-олень 
print (X_train.shape,y_train.shape)

In [ ]:
# Прежде чем приступать к основной работе, убедимся - загруженно именно то, что требовалось:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=[12,10])
for i in range(12):
    plt.subplot(3, 4, i + 1)
    plt.xlabel(class_names[y_train[i, 0]])
    plt.imshow(X_train[i])                 # Входные данные могут быть либо фактическими данными RGB (A), 
                                           # либо двумерными скалярными данными, 
                                           # которые будут визуализированы как псевдоцветное изображение

In [ ]:
# Подготовка данных
# Сейчас каждый пиксель изображения закодирован тройкой чисел (RGB) от 0 до 255. 
# Однако лучше себя показывает подход, где значения входов нейросети распределены недалеко от 0.
# Приведём все данные в диапазон [0, 1] — просто разделим на соответствующий коэффициент:
X_train = X_train / 255
X_val = X_val / 255
X_test = X_test / 255

In [ ]:
# Исполним код ниже для проверки, что все выполнено корректно.
assert np.shape(X_train) == (40000, 32, 32, 3), "data shape should not change"
assert 0.9 <= max(map(np.max, (X_train, X_val, X_test))) <= 1.05
assert 0.0 <= min(map(np.min, (X_train, X_val, X_test))) <= 0.1
assert len(np.unique(X_test / 255.)) > 10, "make sure you casted data to float type" #убедитесь, что привели данные к типу float

In [ ]:
# Архитектура сети
# Для начала реализуем простую нейросеть:

# принимает на вход картинки размера 32 x 32 x 3;
# вытягивает их в вектор (keras.layers.Flatten);
# пропускает через 1 или 2 полносвязных слоя;
# выходной слой отдает вероятности принадлежности к каждому из 10 классов.
import keras
from keras import layers as L
from keras import backend as K

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=3,
                           kernel_size=(5, 5),
                           padding='same',
                           activation='relu',
                           input_shape=X_train.shape[1:]),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    
    tf.keras.layers.Conv2D(filters=16, 
                           kernel_size=(5, 5),
                           padding='valid',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation=None),
    tf.keras.layers.Dropout(0.5), # использование прореживания сократило количество параметров с 35 до 20 тысяч
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(10, activation='softmax')])

model.compile(optimizer='adam',
#               loss=keras.losses.SparseCategoricalCrossentropy(),
              loss='SparseCategoricalCrossentropy',
              metrics=['accuracy'])
# про функции потерь: http://www.100byte.ru/python/loss/loss.html 

In [ ]:
model.summary()

In [ ]:
# Проведем обучение модели
model.fit(X_train, 
          y_train,
          batch_size=32, 
          epochs=6,
          validation_split=0.3)

In [ ]:
# предскажем
y_pred = model.predict(X_val)

In [ ]:
# оценим качество решения на валидационной выборке 
print('Accuracy: %s' % accuracy_score(y_val, y_pred_labels))